# Read ASC files

## Import libraries

In [15]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
from osgeo import gdal
import numpy as np
from osgeo import osr

## Read DENUE file

In [11]:
gpkg_gdf = gpd.read_file('data/colima/agebs.gpkg')
gpkg_gdf = gpkg_gdf[["CVE_AGEB", "geometry"]]
gpkg_gdf.head()

,CVE_AGEB,geometry
0,0676,"POLYGON ((2319182.093 807954.424, 2319286.446 ..."
1,0727,"POLYGON ((2318544.164 807545.878, 2318566.020 ..."
2,1142,"POLYGON ((2316882.335 805042.325, 2317096.142 ..."
3,0411,"POLYGON ((2319801.226 808675.655, 2319836.062 ..."
4,1068,"POLYGON ((2320222.167 809638.735, 2320222.724 ..."


## GDAL

## Ubicación de los archivos ASC

In [4]:
asc_files = glob.glob('data/Rasters/*/*.asc')
asc_files[:5]

['data/Rasters\\Hydraulic\\Depth____0.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10200.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10500.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10800.0.asc',
 'data/Rasters\\Hydraulic\\Depth____11100.0.asc']

## GDAL

In [13]:
# Open the ASC file as a GDAL dataset
asc_ds = gdal.Open(asc_files[0])

### first method

In [10]:
# Get the geotransform and projection of the ASC file
asc_gt = asc_ds.GetGeoTransform()
asc_proj = asc_ds.GetProjection()

# Read the ASC file as a numpy array
asc_arr = asc_ds.ReadAsArray()

# Close the GDAL dataset
asc_ds = None

# Create a GeoDataFrame from the ASC array, geotransform and projection
asc_gdf = gpd.GeoDataFrame.from_features(
    gdal.polygonize(asc_arr, None, options=["8CONNECTED=8"]),
    crs=asc_proj
)

# Set the column name of the ASC values
asc_gdf.columns = ["value", "geometry"]

# Open the gpkg file as a GeoDataFrame
#gpkg_gdf = gpd.read_file("file.gpkg")

# Perform a spatial join to extract the zones that intersect with the ASC file
zones_gdf = gpd.sjoin(asc_gdf, gpkg_gdf, how="inner", op="intersects")

# Save the zones as a new gpkg file
zones_gdf.to_file("zones.gpkg", driver="GPKG")

AttributeError: module 'osgeo.gdal' has no attribute 'polygonize'

### second method

In [16]:
# Get the spatial reference of the dataset and extract the projection information:
proj_info = asc_ds.GetProjection()
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(proj_info)


5

In [17]:
# Extract the geotransform of the dataset:
geotransform = asc_ds.GetGeoTransform()


In [18]:
# Read the data from the ASC file and convert it to a numpy array:
data = asc_ds.ReadAsArray()

In [19]:
# Identify the number of rows and columns in the dataset:
rows, cols = data.shape

In [20]:
# Compute the x and y coordinates for each cell in the dataset:

x_coords = np.arange(cols) * geotransform[1] + geotransform[0]
y_coords = np.arange(rows) * geotransform[5] + geotransform[3]

In [21]:
# Create a meshgrid of the x and y coordinates:

x_mesh, y_mesh = np.meshgrid(x_coords, y_coords)

In [24]:
import proj4

ModuleNotFoundError: No module named 'proj4'

In [23]:
# Convert the x and y coordinates to the projection of the dataset:
x_proj, y_proj = osr.CoordinateTransformation(spatial_ref, spatial_ref).TransformPoints((x_mesh.flatten(), y_mesh.flatten()))[:2]


TypeError: in method 'CoordinateTransformation_TransformPoints', argument 1 of type 'OSRCoordinateTransformationShadow *'